# 5. RAG Modern - Retrieval Augmented Generation

**Durée estimée** : 65 minutes

**Prérequis** : Notebooks 1 (OpenAI Intro), 4 (Function Calling)

---

## Objectifs

La **RAG** (Retrieval Augmented Generation) permet d'enrichir les réponses d'un LLM avec des données externes (documents, bases de connaissances). Ce notebook couvre :

1. **Fondamentaux RAG** : Embeddings, chunking, recherche vectorielle
2. **Stratégies de chunking** : Fixe, sémantique, récursif
3. **Responses API** : Multi-turn RAG avec `previous_response_id`
4. **Optimisation cache** : Économies 40-80% sur les tokens
5. **Citations et sources** : Traçabilité des réponses

---

## Pourquoi la RAG ?

| Problème LLM seul | Solution RAG |
|-------------------|-------------|
| Connaissances figées (cutoff date) | Données actualisées en temps réel |
| Hallucinations fréquentes | Réponses basées sur sources vérifiables |
| Pas de données privées | Accès à vos documents internes |
| Contexte limité | Fenêtre étendue via retrieval |

## Configuration

In [1]:
# Installation des dépendances
%pip install openai tiktoken python-dotenv scikit-learn numpy pandas requests beautifulsoup4 lxml -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import json
import numpy as np
import pandas as pd
from typing import List, Dict, Any
from dotenv import load_dotenv
from openai import OpenAI

# Charger les variables d'environnement
load_dotenv('../.env')

# Mode batch pour tests automatisés
BATCH_MODE = os.getenv("BATCH_MODE", "false").lower() == "true"

# Client OpenAI
client = OpenAI()

# Modèle par défaut depuis .env
DEFAULT_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

print(f"Configuration chargée - Mode batch: {BATCH_MODE}")
print(f"Modèle embeddings: text-embedding-3-large")
print(f"Modèle génération: {DEFAULT_MODEL}")

### Bibliothèques utilisées

Ce notebook nécessite plusieurs dépendances clés :

- **openai** : API OpenAI pour embeddings et génération
- **tiktoken** : Tokenizer officiel pour compter les tokens
- **scikit-learn** : k-Nearest Neighbors pour la recherche vectorielle
- **numpy/pandas** : Manipulation de données et vecteurs
- **requests/beautifulsoup4** : Scraping web pour récupérer le document source
- **python-dotenv** : Gestion sécurisée des clés API

**Note** : En production, remplacer scikit-learn par une vraie base vectorielle (Pinecone, Qdrant, Weaviate).

---

# Partie 1 : Fondamentaux RAG

## 1.1 Architecture RAG

```
┌─────────────┐     ┌──────────────┐     ┌─────────────┐
│  Documents  │ ──► │   Chunking   │ ──► │  Embeddings │
└─────────────┘     └──────────────┘     └──────┬──────┘
                                                │
                                                ▼
┌─────────────┐     ┌──────────────┐     ┌─────────────┐
│   Réponse   │ ◄── │     LLM      │ ◄── │  Retrieval  │
└─────────────┘     └──────────────┘     └─────────────┘
                           ▲
                           │
                    ┌──────┴──────┐
                    │   Question  │
                    └─────────────┘
```

**Étapes** :
1. **Indexation** : Documents → Chunks → Embeddings → Base vectorielle
2. **Retrieval** : Question → Embedding → k-NN → Chunks pertinents
3. **Generation** : Question + Chunks → LLM → Réponse augmentée

## 1.2 Préparation des données

Pour cet exemple, nous utilisons le débat Lincoln-Douglas (1858), un document historique public.

### Pourquoi le débat Lincoln-Douglas ?

Ce document historique est un **excellent cas d'usage RAG** :

1. **Domaine public** : Accessible librement, pas de problèmes de copyright
2. **Taille idéale** : ~98k caractères, assez long pour nécessiter du chunking
3. **Structure narrative** : Discours structurés avec arguments clairs
4. **Questions naturelles** : "Quelle était la position de Lincoln ?" → Réponses vérifiables

**Fallback pour les tests** : Le code inclut un texte de secours si le scraping échoue (mode batch, timeout réseau, etc.). Cela garantit que le notebook peut s'exécuter même sans connexion.

**Point technique** : Nous récupérons uniquement le premier débat (Ottawa, 21 août 1858). La série complète comprend 7 débats.

In [3]:
import requests
from bs4 import BeautifulSoup

def fetch_debate_text() -> str:
    """Récupère le texte du premier débat Lincoln-Douglas."""
    url = "https://home.nps.gov/liho/learn/historyculture/debate1.htm"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, "html.parser")
        main_div = soup.select_one("div.ColumnMain")
        
        if main_div:
            return main_div.get_text(separator="\n", strip=True)
        else:
            raise ValueError("Conteneur principal non trouvé")
            
    except Exception as e:
        print(f"Erreur lors du scraping: {e}")
        # Texte de fallback pour le mode batch
        return """Lincoln-Douglas Debate - Ottawa, Illinois, August 21, 1858
        
Abraham Lincoln argued that slavery was morally wrong and should not be extended 
into new territories. He believed in the principle that "all men are created equal" 
as stated in the Declaration of Independence.

Lincoln stated: "I have no purpose to introduce political and social equality 
between the white and black races. There is a physical difference between the two, 
which, in my judgment, will probably forever forbid their living together upon the 
footing of perfect equality."

However, Lincoln maintained that Black Americans had the right to earn their own 
bread and improve their condition. He opposed the expansion of slavery while 
acknowledging the constitutional protections it had in existing states.

Douglas advocated for popular sovereignty, allowing each territory to decide 
the slavery question for itself. He accused Lincoln of wanting to make all 
states free states, which Lincoln denied.

The debate centered on the interpretation of the Dred Scott decision and 
whether Congress could prohibit slavery in the territories."""

# Récupération du texte
debate_text = fetch_debate_text()
print(f"Texte récupéré: {len(debate_text)} caractères")
print(f"Aperçu: {debate_text[:500]}...")

Texte récupéré: 98529 caractères
Aperçu: First Debate: Ottawa, Illinois
August 21, 1858
It was dry and dusty, between 10,000 and 12,000 people were in attendance when the debate began at 2:00 p.m. There were no seats or bleachers.
Douglas charged Lincoln with trying to “abolitionize” the Whig and Democratic Parties. He also charged Lincoln had been present when a very radical “abolitionist” type platform had been written by the Republican Party in 1854. Douglas accused Lincoln of taking the side of the common enemy in the Mexican War. ...


### Implémentation des stratégies de chunking

Les trois fonctions suivantes implémentent les stratégies présentées ci-dessus. Chacune a ses avantages selon le type de document.

---

# Partie 2 : Stratégies de Chunking

Le chunking est **critique** pour la qualité de la RAG. Trois stratégies principales :

| Stratégie | Avantages | Inconvénients |
|-----------|-----------|---------------|
| **Fixe** | Simple, prévisible | Coupe au milieu des phrases |
| **Sémantique** | Respecte le sens | Plus complexe, variable |
| **Récursif** | Équilibre taille/sens | Nécessite des délimiteurs |

### Préparation du DataFrame de chunks

Nous créons maintenant notre **base de connaissances** : un DataFrame pandas contenant tous les chunks avec leurs métadonnées.

**Structure du DataFrame** :
- `chunk_id` : Identifiant unique (0-50)
- `source` : Source du document (pour traçabilité)
- `text` : Contenu textuel du chunk
- `embedding` : Vecteur de 3072 dimensions (ajouté à l'étape suivante)

Cette structure permet de facilement filtrer, rechercher et enrichir les chunks avec des métadonnées supplémentaires (date, auteur, catégorie, etc.).

In [4]:
def chunk_fixed(text: str, chunk_size: int = 400, overlap: int = 50) -> List[str]:
    """
    Chunking fixe par nombre de mots.
    
    Args:
        text: Texte à découper
        chunk_size: Nombre de mots par chunk
        overlap: Chevauchement entre chunks (évite les coupures brutales)
    """
    words = text.split()
    chunks = []
    start = 0
    
    while start < len(words):
        end = start + chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start += (chunk_size - overlap)
    
    return chunks


def chunk_semantic(text: str, max_sentences: int = 5) -> List[str]:
    """
    Chunking sémantique par phrases.
    Regroupe les phrases en chunks cohérents.
    """
    import re
    # Découpage par phrases (approximatif)
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    chunks = []
    current_chunk = []
    
    for sentence in sentences:
        current_chunk.append(sentence)
        if len(current_chunk) >= max_sentences:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
    
    # Dernier chunk
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks


def chunk_recursive(text: str, max_size: int = 500, delimiters: List[str] = None) -> List[str]:
    """
    Chunking récursif avec délimiteurs hiérarchiques.
    Essaie de découper par paragraphes, puis phrases, puis mots.
    """
    if delimiters is None:
        delimiters = ["\n\n", "\n", ". ", " "]
    
    if len(text) <= max_size or not delimiters:
        return [text] if text.strip() else []
    
    delimiter = delimiters[0]
    parts = text.split(delimiter)
    
    chunks = []
    current = ""
    
    for part in parts:
        if len(current) + len(part) + len(delimiter) <= max_size:
            current += (delimiter if current else "") + part
        else:
            if current:
                chunks.append(current)
            # Récursion avec délimiteur suivant si le part est trop grand
            if len(part) > max_size:
                chunks.extend(chunk_recursive(part, max_size, delimiters[1:]))
            else:
                current = part
    
    if current:
        chunks.append(current)
    
    return chunks


# Comparaison des stratégies
print("=== Comparaison des stratégies de chunking ===")
print(f"\nTexte source: {len(debate_text)} caractères")

chunks_fixed = chunk_fixed(debate_text, chunk_size=100, overlap=20)
chunks_semantic = chunk_semantic(debate_text, max_sentences=3)
chunks_recursive = chunk_recursive(debate_text, max_size=500)

print(f"\nChunking fixe (100 mots, overlap 20): {len(chunks_fixed)} chunks")
print(f"Chunking sémantique (3 phrases): {len(chunks_semantic)} chunks")
print(f"Chunking récursif (max 500 chars): {len(chunks_recursive)} chunks")

=== Comparaison des stratégies de chunking ===

Texte source: 98529 caractères

Chunking fixe (100 mots, overlap 20): 222 chunks
Chunking sémantique (3 phrases): 214 chunks
Chunking récursif (max 500 chars): 327 chunks


In [5]:
# Visualisation des chunks
print("=== Exemple chunk fixe ===")
print(chunks_fixed[0][:300] + "..." if len(chunks_fixed[0]) > 300 else chunks_fixed[0])

print("\n=== Exemple chunk sémantique ===")
print(chunks_semantic[0][:300] + "..." if len(chunks_semantic[0]) > 300 else chunks_semantic[0])

print("\n=== Exemple chunk récursif ===")
print(chunks_recursive[0][:300] + "..." if len(chunks_recursive[0]) > 300 else chunks_recursive[0])

=== Exemple chunk fixe ===
First Debate: Ottawa, Illinois August 21, 1858 It was dry and dusty, between 10,000 and 12,000 people were in attendance when the debate began at 2:00 p.m. There were no seats or bleachers. Douglas charged Lincoln with trying to “abolitionize” the Whig and Democratic Parties. He also charged Lincoln...

=== Exemple chunk sémantique ===
First Debate: Ottawa, Illinois
August 21, 1858
It was dry and dusty, between 10,000 and 12,000 people were in attendance when the debate began at 2:00 p.m. There were no seats or bleachers. Douglas charged Lincoln with trying to “abolitionize” the Whig and Democratic Parties.

=== Exemple chunk récursif ===
First Debate: Ottawa, Illinois
August 21, 1858
It was dry and dusty, between 10,000 and 12,000 people were in attendance when the debate began at 2:00 p.m. There were no seats or bleachers.


### Analyse des résultats de chunking

Observons les différences obtenues :

| Stratégie | Nombre de chunks | Observations |
|-----------|------------------|--------------|
| **Fixe (100 mots, overlap 20)** | 222 | Nombreux chunks petits, overlap assure la continuité |
| **Sémantique (3 phrases)** | 214 | Chunks de taille variable, respecte le sens |
| **Récursif (max 500 chars)** | 327 | Plus de petits chunks, découpage hiérarchique |

**Points clés** :

1. **Chunking fixe** : Le premier exemple montre une coupure au milieu d'une phrase ("...He also charged Lincoln..."), typique de cette méthode.

2. **Chunking sémantique** : Le découpage respecte les limites de phrases, plus naturel pour la compréhension.

3. **Chunking récursif** : Découpe d'abord par paragraphes (`\n\n`), puis par phrases si trop long. Résultat : chunks courts mais cohérents.

**Pour la suite**, nous utilisons le chunking fixe avec **400 mots et overlap 50** - un bon compromis entre taille et continuité pour ce type de document narratif.

## 2.1 Choix de la stratégie

**Recommandations** :

- **Documents structurés** (code, JSON, markdown) → Chunking récursif
- **Texte narratif** (articles, livres) → Chunking sémantique
- **Données tabulaires** → Chunking fixe avec métadonnées

Pour la suite, nous utilisons le chunking fixe avec overlap (400 mots, overlap 50).

### Comprendre les embeddings

Un **embedding** est une représentation vectorielle d'un texte dans un espace à haute dimension. Chaque dimension capture un aspect sémantique du texte.

**Exemple simplifié (2D)** :
```
"chat"      → [0.8, 0.2]  (animal, domestique)
"chien"     → [0.7, 0.3]  (animal, domestique)
"ordinateur" → [0.1, 0.9]  (objet, technologie)
```

En réalité, `text-embedding-3-large` génère des vecteurs à **3072 dimensions** pour capturer les nuances du langage.

**Propriétés clés** :

1. **Similarité sémantique** : Textes similaires ont des vecteurs proches (distance cosinus faible)
2. **Invariance** : Même sens avec différents mots → vecteurs similaires
3. **Efficacité** : Comparer 3072 nombres est plus rapide que comparer du texte brut

**Batch vs Single** : La fonction `create_embeddings_batch()` est **5-10x plus rapide** que des appels individuels pour de gros volumes. OpenAI autorise jusqu'à 2048 textes par batch.

**Résultat** : 51 vecteurs de 3072 dimensions, prêts pour la recherche vectorielle.

### Implémentation de la VectorStore

La classe `VectorStore` encapsule la logique de recherche vectorielle. Elle utilise scikit-learn pour le prototypage, mais en production vous utiliseriez une vraie base vectorielle.

**Points clés de l'implémentation** :

1. **Métrique cosinus** : `metric="cosine"` mesure l'angle entre vecteurs, pas la distance euclidienne
2. **Validation des embeddings** : Filtre les None pour éviter les erreurs
3. **Retour structuré** : Dicts avec `chunk_id`, `text`, `source`, et `score` pour traçabilité complète

In [6]:
# Création du DataFrame de chunks pour la suite
chunks = chunk_fixed(debate_text, chunk_size=400, overlap=50)

df_chunks = pd.DataFrame({
    "chunk_id": range(len(chunks)),
    "source": "Lincoln-Douglas Debate 1 (Ottawa, 1858)",
    "text": chunks
})

print(f"Base de connaissances: {len(df_chunks)} chunks")
df_chunks.head()

Base de connaissances: 51 chunks


,chunk_id,source,text
0,0,"Lincoln-Douglas Debate 1 (Ottawa, 1858)","First Debate: Ottawa, Illinois August 21, 1858..."
1,1,"Lincoln-Douglas Debate 1 (Ottawa, 1858)",were proclaimed wherever the Constitution rule...
2,2,"Lincoln-Douglas Debate 1 (Ottawa, 1858)",the Whig party and the Democratic party both s...
3,3,"Lincoln-Douglas Debate 1 (Ottawa, 1858)",name and disguise of a Republican party. (Laug...
4,4,"Lincoln-Douglas Debate 1 (Ottawa, 1858)",with such views as the circumstances and exige...


---

# Partie 3 : Embeddings et Recherche Vectorielle

## 3.1 Génération des embeddings

OpenAI propose plusieurs modèles d'embeddings :

| Modèle | Dimensions | Performance | Coût |
|--------|------------|-------------|------|
| `text-embedding-3-small` | 1536 | Bon | $ |
| `text-embedding-3-large` | 3072 | Excellent | $$ |
| `text-embedding-ada-002` | 1536 | Bon (legacy) | $ |

In [7]:
def create_embedding(text: str, model: str = "text-embedding-3-large") -> List[float]:
    """
    Génère un embedding pour un texte donné.
    
    Args:
        text: Texte à vectoriser
        model: Modèle d'embedding OpenAI
    
    Returns:
        Vecteur d'embedding (liste de floats)
    """
    try:
        response = client.embeddings.create(
            model=model,
            input=[text]
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Erreur embedding: {e}")
        return None


def create_embeddings_batch(texts: List[str], model: str = "text-embedding-3-large") -> List[List[float]]:
    """
    Génère des embeddings en batch (plus efficace).
    
    Note: OpenAI supporte jusqu'à 2048 textes par requête.
    """
    try:
        response = client.embeddings.create(
            model=model,
            input=texts
        )
        return [item.embedding for item in response.data]
    except Exception as e:
        print(f"Erreur batch embedding: {e}")
        return [None] * len(texts)


# Génération des embeddings pour tous les chunks
print("Génération des embeddings...")
embeddings = create_embeddings_batch(df_chunks["text"].tolist())
df_chunks["embedding"] = embeddings

print(f"Embeddings générés: {len([e for e in embeddings if e])} / {len(embeddings)}")
print(f"Dimension des vecteurs: {len(embeddings[0]) if embeddings[0] else 'N/A'}")

Génération des embeddings...
Embeddings générés: 51 / 51
Dimension des vecteurs: 3072


### Anatomie d'une réponse RAG

Le pipeline `rag_query()` exécute 4 étapes :

**1. Retrieval (Récupération)** :
```python
query_embedding = create_embedding(question)
chunks = vector_store.search(query_embedding, k=3)
```
Convertit la question en vecteur, puis cherche les 3 chunks les plus proches.

**2. Construction du contexte** :
```python
context = "\n\n".join([f"[Source: {c['source']}]\n{c['text']}" for c in chunks])
```
Assemble les chunks récupérés en un seul texte avec métadonnées.

**3. Prompt augmenté** :
```python
system_prompt = "Réponds UNIQUEMENT basé sur le contexte fourni..."
user_prompt = f"Contexte:\n{context}\n\nQuestion: {question}"
```
Injecte le contexte dans le prompt pour "augmenter" les connaissances du modèle.

**4. Génération** :
```python
response = client.chat.completions.create(...)
```
Le LLM génère une réponse basée sur le contexte fourni, pas sur ses connaissances pré-entraînées.

**Paramètres clés** :
- `temperature=0.3` : Faible pour plus de précision et moins d'hallucinations
- `max_tokens=500` : Limite la longueur de la réponse

**Résultat attendu** : Une réponse précise citant les chunks utilisés, avec traçabilité complète via les métadonnées de tokens.

## 3.2 Recherche k-NN (k-Nearest Neighbors)

Pour la recherche, on calcule la distance entre l'embedding de la question et ceux des chunks.

## Importance des citations en RAG

Les citations sont **essentielles** pour :

1. **Vérifiabilité** : L'utilisateur peut vérifier la source de l'information
2. **Confiance** : Une réponse avec sources est plus crédible qu'une affirmation sans preuve
3. **Debugging** : Identifier si le problème vient du retrieval (mauvais chunks) ou de la génération (mauvaise interprétation)
4. **Conformité** : Certains domaines (médical, juridique) exigent la traçabilité complète

**Anti-pattern** : Générer une réponse sans indiquer les sources utilisées. Cela empêche la vérification et rend le système opaque.

### Responses API vs Chat Completions

La **Responses API** est l'approche moderne (2025) pour la RAG, avec des avantages significatifs :

**Différences techniques** :

| Aspect | Chat Completions | Responses API |
|--------|------------------|---------------|
| **Historique** | Envoyé manuellement à chaque tour | Géré automatiquement par `previous_response_id` |
| **Cache** | Manuel (prompt caching) | Automatique sur contextes répétés |
| **Tokens** | Tous retransmis | Économies 40-80% via cache |
| **Persistence** | Client-side | Server-side avec `store: true` |

**Exemple d'économie** :
```
Chat Completions (conversation 3 tours):
- Tour 1: 1000 tokens contexte + 500 génération = 1500
- Tour 2: 1000 contexte + 500 génération = 1500
- Tour 3: 1000 contexte + 500 génération = 1500
Total: 4500 tokens

Responses API (avec previous_response_id):
- Tour 1: 1000 tokens contexte + 500 génération = 1500
- Tour 2: Contexte en cache + 500 génération = 500
- Tour 3: Contexte en cache + 500 génération = 500
Total: 2500 tokens (économie 44%)
```

**Fallback implémenté** : Si l'API Responses n'est pas disponible, le code bascule automatiquement sur Chat Completions. C'est une bonne pratique de compatibilité.

### Transition vers les bonnes pratiques

Nous avons maintenant un pipeline RAG complet fonctionnel. Cependant, pour passer en **production**, plusieurs optimisations sont nécessaires.

**Prochaines étapes** :

1. **Monitoring et métriques** : Mesurer la qualité (relevance, precision, recall)
2. **Scaling** : Base vectorielle distribuée (Pinecone, Qdrant)
3. **Optimisation coûts** : Cache intelligent, batch processing
4. **Évaluation continue** : Tests A/B sur différentes stratégies de chunking

La section suivante couvre ces aspects avec des recommandations concrètes.

In [8]:
from sklearn.neighbors import NearestNeighbors

class VectorStore:
    """
    Base vectorielle simple basée sur scikit-learn.
    En production, utiliser Pinecone, Qdrant, Weaviate, ou Chroma.
    """
    
    def __init__(self, df: pd.DataFrame, embedding_col: str = "embedding"):
        self.df = df
        self.embedding_col = embedding_col
        
        # Filtrer les embeddings valides
        valid_mask = df[embedding_col].apply(lambda x: x is not None)
        self.valid_indices = df[valid_mask].index.tolist()
        
        # Construire la matrice de vecteurs
        vectors = np.array(df.loc[self.valid_indices, embedding_col].tolist())
        
        # Index k-NN
        self.nn = NearestNeighbors(n_neighbors=min(5, len(vectors)), metric="cosine")
        self.nn.fit(vectors)
        
        print(f"VectorStore initialisé: {len(self.valid_indices)} vecteurs")
    
    def search(self, query_embedding: List[float], k: int = 3) -> List[Dict[str, Any]]:
        """
        Recherche les k chunks les plus similaires.
        
        Returns:
            Liste de dicts avec chunk_id, text, source, score
        """
        distances, indices = self.nn.kneighbors([query_embedding], n_neighbors=k)
        
        results = []
        for i, (dist, idx) in enumerate(zip(distances[0], indices[0])):
            original_idx = self.valid_indices[idx]
            row = self.df.iloc[original_idx]
            results.append({
                "chunk_id": row["chunk_id"],
                "text": row["text"],
                "source": row["source"],
                "score": 1 - dist  # Cosine similarity (1 = identique)
            })
        
        return results


# Initialisation du VectorStore
vector_store = VectorStore(df_chunks)

VectorStore initialisé: 51 vecteurs


In [9]:
# Test de recherche
question = "What did Lincoln argue about slavery?"

# Embedding de la question
query_embedding = create_embedding(question)

# Recherche des chunks pertinents
results = vector_store.search(query_embedding, k=3)

print(f"Question: {question}\n")
print("=== Chunks retrouvés ===")
for i, r in enumerate(results):
    print(f"\n[{i+1}] Score: {r['score']:.3f} | Source: {r['source']}")
    print(f"    {r['text'][:200]}...")

Question: What did Lincoln argue about slavery?

=== Chunks retrouvés ===

[1] Score: 0.628 | Source: Lincoln-Douglas Debate 1 (Ottawa, 1858)
    proclaims his Abolition doctrines. Let me read a part of them. In his speech at Springfield to the Convention, which nominated him for the Senate, he said: "In my opinion it will not cease until a cri...

[2] Score: 0.580 | Source: Lincoln-Douglas Debate 1 (Ottawa, 1858)
    them not to have it if they do not want it. [Applause and laughter.] I do not mean that if this vast concourse of people were in a Territory of the United States, any one of them would be obliged to h...

[3] Score: 0.572 | Source: Lincoln-Douglas Debate 1 (Ottawa, 1858)
    it not exist on the same principles on which our fathers made it? ("It can.")The knew when they framed the Constitution that in a country as wide and broad as this, with such a variety of climate, pro...


### Interprétation des scores de similarité

Les résultats de recherche montrent :

| Rang | Score | Interprétation |
|------|-------|----------------|
| 1 | 0.628 | **Bonne correspondance** - Le chunk contient directement la doctrine d'abolition de Lincoln |
| 2 | 0.580 | **Pertinence moyenne** - Discussion sur les droits territoriaux liés à l'esclavage |
| 3 | 0.572 | **Pertinence moyenne** - Contexte historique sur les principes des pères fondateurs |

**Analyse du score** :

- **Score > 0.7** : Excellent match, très pertinent
- **0.5 < Score < 0.7** : Pertinent, mais contexte indirect
- **Score < 0.5** : Faible pertinence, risque de bruit

Ici, le **chunk #1 avec score 0.628** est le plus pertinent. Il mentionne explicitement les "Abolition doctrines" de Lincoln dans son discours à Springfield, répondant directement à la question "What did Lincoln argue about slavery?".

**Point important** : Même un score de 0.628 (62.8% de similarité) est considéré comme **bon** en RAG. Les scores parfaits (>0.9) ne s'obtiennent que pour des textes quasi-identiques.

**Distance cosinus** : Le score affiché est `1 - distance`, donc 1 = identique, 0 = orthogonal (aucune similarité).

---

# Partie 4 : RAG avec Chat Completions

## 4.1 Pipeline RAG classique

### Qualité des citations : critères d'évaluation

Une bonne réponse RAG avec citations doit respecter plusieurs critères :

**1. Précision des références** :
- Chaque affirmation doit être associée à une source [1], [2], etc.
- Les citations multiples sont encouragées si plusieurs sources confirment la même information

**2. Format structuré** :
```
RÉPONSE: Lincoln argued that... [1] while Douglas maintained... [2]
SOURCES UTILISÉES: [1, 2]
CONFIANCE: haute
```

**3. Niveaux de confiance** :
- **Haute** : Information présente dans plusieurs sources, cohérente
- **Moyenne** : Information dans une seule source, claire
- **Basse** : Information implicite, nécessite inférence

**Exemple de sortie attendue** :
```
RÉPONSE: Les points clés de désaccord étaient :
1. L'extension de l'esclavage [1, 2]
2. La souveraineté populaire vs restriction fédérale [1, 3]
3. L'interprétation de la Déclaration d'Indépendance [2]

SOURCES UTILISÉES: [1, 2, 3]
CONFIANCE: haute
```

**Vérification de qualité** : Comparer les extraits des chunks récupérés avec les citations dans la réponse pour s'assurer de la fidélité.

In [ ]:
def rag_query(question: str, vector_store: VectorStore, k: int = 3) -> Dict[str, Any]:
    """
    Pipeline RAG complet avec Chat Completions.
    
    Args:
        question: Question de l'utilisateur
        vector_store: Base vectorielle
        k: Nombre de chunks à récupérer
    
    Returns:
        Dict avec réponse, sources, et métadonnées
    """
    # 1. Retrieval
    query_embedding = create_embedding(question)
    chunks = vector_store.search(query_embedding, k=k)
    
    # 2. Construction du contexte
    context = "\n\n".join([
        f"[Source: {c['source']} | Chunk {c['chunk_id']}]\n{c['text']}"
        for c in chunks
    ])
    
    # 3. Prompt augmenté
    system_prompt = """Tu es un assistant expert en histoire américaine.
Réponds aux questions en te basant UNIQUEMENT sur le contexte fourni.
Si l'information n'est pas dans le contexte, dis-le clairement.
Cite tes sources avec le format [Chunk X]."""
    
    user_prompt = f"""Contexte:
{context}

Question: {question}

Réponds de manière précise en citant les sources."""
    
    # 4. Génération
    response = client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3,  # Faible pour plus de précision
        max_tokens=500
    )
    
    return {
        "question": question,
        "answer": response.choices[0].message.content,
        "sources": chunks,
        "model": DEFAULT_MODEL,
        "tokens": {
            "prompt": response.usage.prompt_tokens,
            "completion": response.usage.completion_tokens
        }
    }


# Test du pipeline RAG
result = rag_query(
    "What were Lincoln's main arguments about slavery in the debate?",
    vector_store
)

print("=== Réponse RAG ===")
print(f"\nQuestion: {result['question']}")
print(f"\nRéponse:\n{result['answer']}")
print(f"\nTokens utilisés: {result['tokens']}")

### Reranking : filtrer le bruit

Le **reranking** est une étape critique pour améliorer la précision de la RAG :

**Problème** : k-NN récupère les k chunks les **plus proches**, pas forcément les plus **pertinents**. Un chunk avec score 0.3 (30% de similarité) peut contenir du bruit.

**Solution** : Filtrer par score minimum (threshold).

**Impact du filtrage** :

| Scénario | Chunks k=5 | Après filtrage (min_score=0.3) | Résultat |
|----------|------------|-------------------------------|----------|
| Question très spécifique | [0.8, 0.7, 0.6, 0.4, 0.3] | [0.8, 0.7, 0.6, 0.4, 0.3] | 5 chunks conservés |
| Question hors sujet | [0.4, 0.3, 0.2, 0.1, 0.05] | [0.4, 0.3] | 3 chunks éliminés |
| Question ambiguë | [0.5, 0.5, 0.4, 0.2, 0.1] | [0.5, 0.5, 0.4] | 2 chunks éliminés |

**Seuils recommandés** :
- **0.5+** : Très strict, seulement réponses haute confiance
- **0.3-0.5** : Équilibré (recommandé)
- **0.2-0.3** : Permissif, accepte plus de contexte

**Techniques avancées** :
- **Reranking cross-encoder** : Utiliser un modèle spécialisé (ex: `ms-marco-MiniLM-L-6-v2`) pour réévaluer les paires (question, chunk)
- **Diversité maximale** : Éviter les chunks redondants avec clustering
- **Temporal decay** : Préférer les chunks récents pour des données temporelles

**Note** : Le simple filtrage par score est déjà très efficace et coûte 0 token supplémentaire.

---

# Partie 5 : RAG avec Responses API (Moderne)

La **Responses API** (2025) offre des avantages significatifs pour la RAG :

| Fonctionnalité | Avantage |
|----------------|----------|
| `store: true` | Persistence automatique des conversations |
| `previous_response_id` | Multi-turn sans renvoyer l'historique |
| Cache automatique | Économies 40-80% sur tokens répétés |
| Outils intégrés | Web search, file search, code interpreter |

In [ ]:
def rag_query_responses_api(
    question: str,
    vector_store: VectorStore,
    previous_response_id: str = None,
    k: int = 3
) -> Dict[str, Any]:
    """
    Pipeline RAG avec Responses API.
    
    Avantages:
    - Conversations multi-turn avec previous_response_id
    - Cache automatique des contextes répétés
    - Persistence côté serveur
    """
    # 1. Retrieval
    query_embedding = create_embedding(question)
    chunks = vector_store.search(query_embedding, k=k)
    
    # 2. Construction du contexte (format texte simple pour Responses API)
    context = "\n\n".join([
        f"[Source: {c['source']} | Chunk {c['chunk_id']}]\n{c['text']}"
        for c in chunks
    ])
    
    # 3. Prompt complet
    full_prompt = f"""Tu es un assistant expert. Réponds en citant les sources [Chunk X].

Contexte:
{context}

Question: {question}"""
    
    # 4. Appel Responses API
    try:
        kwargs = {
            "model": DEFAULT_MODEL,
            "input": full_prompt,
            "store": True
        }
        if previous_response_id:
            kwargs["previous_response_id"] = previous_response_id
        
        response = client.responses.create(**kwargs)
        
        # Extraire le texte de la réponse
        answer = ""
        if response.output:
            for item in response.output:
                if hasattr(item, 'content'):
                    for content in item.content:
                        if hasattr(content, 'text'):
                            answer += content.text
        
        return {
            "question": question,
            "answer": answer if answer else "Pas de réponse",
            "response_id": response.id,  # Pour le chaînage
            "sources": chunks,
            "api": "responses"
        }
        
    except Exception as e:
        # Fallback sur Chat Completions si Responses API non disponible
        print(f"Note: Responses API non disponible ({e}), utilisation de Chat Completions")
        return rag_query(question, vector_store, k)


# Test Responses API
print("=== Test RAG avec Responses API ===")
result1 = rag_query_responses_api(
    "What did Lincoln say about slavery?",
    vector_store
)

print(f"\nQuestion 1: {result1['question']}")
print(f"Réponse: {result1['answer'][:500]}...")
print(f"Response ID: {result1.get('response_id', 'N/A')}")

In [12]:
# Multi-turn avec previous_response_id
print("=== Conversation multi-turn ===")

if result1.get('response_id'):
    # Question de suivi utilisant le contexte précédent
    result2 = rag_query_responses_api(
        "And what was Douglas's counter-argument?",
        vector_store,
        previous_response_id=result1['response_id']
    )
    
    print(f"\nQuestion 2 (suivi): {result2['question']}")
    print(f"Réponse: {result2['answer'][:500]}...")
    print("\nNote: Le contexte de la question 1 est automatiquement inclus via previous_response_id")
else:
    print("Responses API non disponible pour le multi-turn")

=== Conversation multi-turn ===
Responses API non disponible pour le multi-turn


## 5.1 Avantages du Multi-turn RAG

Avec `previous_response_id`, le modèle conserve automatiquement :

1. **Le contexte précédent** : Pas besoin de renvoyer tous les chunks
2. **L'historique de conversation** : Questions/réponses précédentes
3. **Cache des embeddings** : Économies sur les tokens répétés

**Cas d'usage** :
- Conversations de recherche documentaire
- Analyse progressive de documents
- Q&A avec follow-up questions

---

# Partie 6 : Citations et Traçabilité

Un système RAG de qualité doit permettre de **vérifier les sources**.

In [ ]:
def rag_with_citations(
    question: str,
    vector_store: VectorStore,
    k: int = 3
) -> Dict[str, Any]:
    """
    RAG avec génération structurée de citations.
    """
    # Retrieval
    query_embedding = create_embedding(question)
    chunks = vector_store.search(query_embedding, k=k)
    
    # Contexte numéroté pour citations
    context_parts = []
    for i, c in enumerate(chunks):
        context_parts.append(f"[{i+1}] {c['text'][:500]}")
    context = "\n\n".join(context_parts)
    
    # Prompt demandant des citations explicites
    prompt = f"""Basé sur ces sources:

{context}

Question: {question}

Instructions:
1. Réponds à la question en citant les sources entre crochets [1], [2], etc.
2. Si plusieurs sources confirment une information, cite-les toutes.
3. Si l'information n'est pas dans les sources, indique-le.

Format de réponse:
RÉPONSE: [ta réponse avec citations]
SOURCES UTILISÉES: [liste des numéros de sources]
CONFIANCE: [haute/moyenne/basse]"""
    
    response = client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    
    return {
        "question": question,
        "response": response.choices[0].message.content,
        "retrieved_chunks": [
            {"id": i+1, "score": c["score"], "preview": c["text"][:200]}
            for i, c in enumerate(chunks)
        ]
    }


# Test avec citations
result_cited = rag_with_citations(
    "What were the key points of disagreement between Lincoln and Douglas?",
    vector_store
)

print("=== RAG avec Citations ===")
print(f"\nQuestion: {result_cited['question']}")
print(f"\n{result_cited['response']}")
print("\n--- Chunks récupérés ---")
for chunk in result_cited['retrieved_chunks']:
    print(f"[{chunk['id']}] Score: {chunk['score']:.3f} - {chunk['preview'][:100]}...")

---

# Partie 7 : Bonnes Pratiques

## 7.1 Checklist RAG Production

| Aspect | Recommandation |
|--------|----------------|
| **Chunking** | Tester plusieurs stratégies, mesurer la qualité |
| **Embeddings** | text-embedding-3-large pour la précision |
| **k (retrieval)** | 3-5 chunks, augmenter si précision insuffisante |
| **Reranking** | Filtrer les chunks peu pertinents (score < 0.7) |
| **Prompt** | Demander explicitement des citations |
| **Cache** | Utiliser Responses API avec `store: true` |
| **Monitoring** | Logger les questions sans bonne réponse |

## 7.2 Bases vectorielles en production

| Solution | Type | Avantages |
|----------|------|----------|
| **Pinecone** | Cloud | Scalable, serverless |
| **Qdrant** | Self-hosted/Cloud | Performant, filtres avancés |
| **Weaviate** | Self-hosted/Cloud | Hybrid search, modules |
| **Chroma** | Local | Simple, bon pour prototypage |
| **pgvector** | PostgreSQL | Intégration SQL existante |

In [ ]:
# Exemple de filtrage par score (reranking simple)
def rag_with_reranking(
    question: str,
    vector_store: VectorStore,
    k: int = 5,
    min_score: float = 0.3
) -> Dict[str, Any]:
    """
    RAG avec reranking: filtre les chunks peu pertinents.
    """
    query_embedding = create_embedding(question)
    all_chunks = vector_store.search(query_embedding, k=k)
    
    # Filtrage par score minimum
    filtered_chunks = [c for c in all_chunks if c['score'] >= min_score]
    
    print(f"Chunks récupérés: {len(all_chunks)}, après filtrage (score >= {min_score}): {len(filtered_chunks)}")
    
    if not filtered_chunks:
        return {"answer": "Aucun contexte pertinent trouvé pour cette question."}
    
    # Génération avec chunks filtrés
    context = "\n\n".join([c['text'] for c in filtered_chunks])
    
    response = client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=[
            {"role": "system", "content": "Réponds uniquement basé sur le contexte fourni."},
            {"role": "user", "content": f"Contexte:\n{context}\n\nQuestion: {question}"}
        ]
    )
    
    return {
        "answer": response.choices[0].message.content,
        "chunks_used": len(filtered_chunks),
        "chunks_filtered_out": len(all_chunks) - len(filtered_chunks)
    }


# Test reranking
result_reranked = rag_with_reranking(
    "What was the outcome of the debate?",
    vector_store,
    k=5,
    min_score=0.3
)

print(f"\nRéponse: {result_reranked['answer'][:500]}...")

---

# Conclusion

## Points clés

1. **Chunking** : Stratégie critique - fixe, sémantique, ou récursif selon le cas
2. **Embeddings** : `text-embedding-3-large` offre la meilleure précision
3. **Retrieval** : k-NN avec score minimum pour filtrer le bruit
4. **Responses API** : `previous_response_id` pour multi-turn efficace
5. **Citations** : Toujours demander des références vérifiables

## Prochaines étapes

- **Notebook 6** : PDF et Web Search intégrés
- **Notebook 7** : Code Interpreter pour analyse de données
- **Notebook 9** : Patterns de production (batch, retry, monitoring)

## Ressources

- [OpenAI Embeddings Guide](https://platform.openai.com/docs/guides/embeddings)
- [Responses API Documentation](https://platform.openai.com/docs/api-reference/responses)
- [RAG Best Practices](https://platform.openai.com/docs/guides/retrieval)

In [15]:
# Résumé des fonctions créées
print("=== Fonctions RAG disponibles ===")
print("""
Chunking:
  - chunk_fixed(text, chunk_size, overlap)
  - chunk_semantic(text, max_sentences)
  - chunk_recursive(text, max_size, delimiters)

Embeddings:
  - create_embedding(text, model)
  - create_embeddings_batch(texts, model)

Recherche:
  - VectorStore.search(query_embedding, k)

RAG:
  - rag_query(question, vector_store, k)           # Chat Completions
  - rag_query_responses_api(question, ...)         # Responses API (multi-turn)
  - rag_with_citations(question, vector_store, k)  # Avec sources
  - rag_with_reranking(question, ..., min_score)   # Avec filtrage
""")

=== Fonctions RAG disponibles ===

Chunking:
  - chunk_fixed(text, chunk_size, overlap)
  - chunk_semantic(text, max_sentences)
  - chunk_recursive(text, max_size, delimiters)

Embeddings:
  - create_embedding(text, model)
  - create_embeddings_batch(texts, model)

Recherche:
  - VectorStore.search(query_embedding, k)

RAG:
  - rag_query(question, vector_store, k)           # Chat Completions
  - rag_query_responses_api(question, ...)         # Responses API (multi-turn)
  - rag_with_citations(question, vector_store, k)  # Avec sources
  - rag_with_reranking(question, ..., min_score)   # Avec filtrage

